Combinacion de resultados - Ensemble

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

print("Librerías importadas correctamente")

Librerías importadas correctamente


In [2]:
# Cargar los archivos de predicciones
print("Cargando archivos de predicciones...")

# Cargar predicciones de AutoGluon
pred_autogluon = pd.read_csv('data/pred_autogluon_01.csv')
print(f"AutoGluon - Shape: {pred_autogluon.shape}")
print("AutoGluon - Primeras filas:")
print(pred_autogluon.head())

print("\n" + "="*50 + "\n")

# Cargar predicciones de Regresión Lineal
pred_regresion = pd.read_csv('data/pred_regresion_lineal01.csv')
print(f"Regresión Lineal - Shape: {pred_regresion.shape}")
print("Regresión Lineal - Primeras filas:")
print(pred_regresion.head())

Cargando archivos de predicciones...
AutoGluon - Shape: (780, 2)
AutoGluon - Primeras filas:
   product_id           tn
0       20001  1322.782727
1       20002  1076.158858
2       20003   696.172870
3       20004   519.217793
4       20005   501.010270


Regresión Lineal - Shape: (780, 2)
Regresión Lineal - Primeras filas:
   product_id           tn
0       20001  1162.707525
1       20002  1183.640604
2       20003   684.763931
3       20004   580.484961
4       20005   563.560780


In [3]:
# Verificar que ambos archivos tienen los mismos productos
print("Verificando consistencia de datos...")
print(f"Productos únicos en AutoGluon: {pred_autogluon['product_id'].nunique()}")
print(f"Productos únicos en Regresión Lineal: {pred_regresion['product_id'].nunique()}")

# Verificar si tienen exactamente los mismos product_ids
productos_autogluon = set(pred_autogluon['product_id'])
productos_regresion = set(pred_regresion['product_id'])

if productos_autogluon == productos_regresion:
    print("✅ Ambos archivos tienen exactamente los mismos productos")
else:
    print("⚠️ Los archivos NO tienen los mismos productos")
    en_autogluon_no_en_regresion = productos_autogluon - productos_regresion
    en_regresion_no_en_autogluon = productos_regresion - productos_autogluon
    
    if en_autogluon_no_en_regresion:
        print(f"Productos en AutoGluon pero no en Regresión: {len(en_autogluon_no_en_regresion)}")
    if en_regresion_no_en_autogluon:
        print(f"Productos en Regresión pero no en AutoGluon: {len(en_regresion_no_en_autogluon)}")

print(f"\nProductos en común: {len(productos_autogluon & productos_regresion)}")

Verificando consistencia de datos...
Productos únicos en AutoGluon: 780
Productos únicos en Regresión Lineal: 780
✅ Ambos archivos tienen exactamente los mismos productos

Productos en común: 780


In [4]:
# Crear ensemble con ponderación específica
print("Creando ensemble ponderado (40% AutoGluon + 60% Regresión)...")

# Renombrar las columnas tn para distinguir los modelos
pred_autogluon_renamed = pred_autogluon.rename(columns={'tn': 'tn_autogluon'})
pred_regresion_renamed = pred_regresion.rename(columns={'tn': 'tn_regresion'})

# Hacer merge por product_id
ensemble_data = pd.merge(
    pred_autogluon_renamed, 
    pred_regresion_renamed, 
    on='product_id', 
    how='inner'
)

print(f"Datos después del merge: {ensemble_data.shape}")
print("\nPrimeras filas del merge:")
print(ensemble_data.head())

# Calcular el promedio ponderado de las predicciones
# 25% AutoGluon + 75% Regresión Lineal
peso_autogluon = 0.25
peso_regresion = 0.75

ensemble_data['tn'] = (ensemble_data['tn_autogluon'] * peso_autogluon + 
                       ensemble_data['tn_regresion'] * peso_regresion)

print(f"\nEstadísticas del ensemble ponderado (25% AutoGluon + 75% Regresión):")
print(f"Promedio de tn: {ensemble_data['tn'].mean():.2f}")
print(f"Mediana de tn: {ensemble_data['tn'].median():.2f}")
print(f"Std de tn: {ensemble_data['tn'].std():.2f}")

# Mostrar comparación de los primeros productos
print(f"\nComparación de predicciones para los primeros 5 productos:")
print(f"(AutoGluon: {peso_autogluon*100:.0f}%, Regresión: {peso_regresion*100:.0f}%)")
comparison = ensemble_data[['product_id', 'tn_autogluon', 'tn_regresion', 'tn']].head()
print(comparison)

Creando ensemble ponderado (40% AutoGluon + 60% Regresión)...
Datos después del merge: (780, 3)

Primeras filas del merge:
   product_id  tn_autogluon  tn_regresion
0       20001   1322.782727   1162.707525
1       20002   1076.158858   1183.640604
2       20003    696.172870    684.763931
3       20004    519.217793    580.484961
4       20005    501.010270    563.560780

Estadísticas del ensemble ponderado (25% AutoGluon + 75% Regresión):
Promedio de tn: 35.82
Mediana de tn: 9.19
Std de tn: 92.48

Comparación de predicciones para los primeros 5 productos:
(AutoGluon: 25%, Regresión: 75%)
   product_id  tn_autogluon  tn_regresion           tn
0       20001   1322.782727   1162.707525  1202.726326
1       20002   1076.158858   1183.640604  1156.770168
2       20003    696.172870    684.763931   687.616165
3       20004    519.217793    580.484961   565.168169
4       20005    501.010270    563.560780   547.923153


In [5]:
# Crear el archivo final con solo product_id y tn promediado
print("Preparando datos finales para guardar...")

# Seleccionar solo las columnas necesarias para el archivo final
resultado_final = ensemble_data[['product_id', 'tn']].copy()

# Verificar que no hay valores nulos
print(f"Valores nulos en el resultado final: {resultado_final.isnull().sum().sum()}")
print(f"Shape del resultado final: {resultado_final.shape}")

print("\nPrimeras 10 filas del resultado final:")
print(resultado_final.head(10))

print(f"\nÚltimas 5 filas del resultado final:")
print(resultado_final.tail())

Preparando datos finales para guardar...
Valores nulos en el resultado final: 0
Shape del resultado final: (780, 2)

Primeras 10 filas del resultado final:
   product_id           tn
0       20001  1202.726326
1       20002  1156.770168
2       20003   687.616165
3       20004   565.168169
4       20005   547.923153
5       20006   475.010816
6       20007   392.568311
7       20008   414.211320
8       20009   460.561362
9       20010   409.796967

Últimas 5 filas del resultado final:
     product_id        tn
775       20962  3.479425
776       20975  3.196880
777       20995  3.052063
778       21087  1.231390
779       21214  0.745201


In [6]:
# Guardar el archivo final
archivo_salida = "data/pred_autogluon_rl_03.csv"

print(f"Guardando resultado en: {archivo_salida}")

# Guardar el archivo
resultado_final.to_csv(archivo_salida, index=False)

print(f"✅ Archivo guardado exitosamente: {archivo_salida}")
print(f"Total de productos en el ensemble: {len(resultado_final)}")

# Verificar que el archivo se guardó correctamente
import os
if os.path.exists(archivo_salida):
    tamaño_archivo = os.path.getsize(archivo_salida)
    print(f"Tamaño del archivo: {tamaño_archivo} bytes")
    
    # Leer las primeras líneas del archivo guardado para verificar
    verificacion = pd.read_csv(archivo_salida, nrows=3)
    print(f"\nVerificación - Primeras 3 líneas del archivo guardado:")
    print(verificacion)
else:
    print("⚠️ Error: No se pudo crear el archivo")

Guardando resultado en: data/pred_autogluon_rl_03.csv
✅ Archivo guardado exitosamente: data/pred_autogluon_rl_03.csv
Total de productos en el ensemble: 780
Tamaño del archivo: 19910 bytes

Verificación - Primeras 3 líneas del archivo guardado:
   product_id           tn
0       20001  1202.726326
1       20002  1156.770168
2       20003   687.616165


In [7]:
# Análisis final del ensemble
print("RESUMEN DEL ENSEMBLE")
print("="*50)

# Estadísticas descriptivas de cada modelo
print("Estadísticas de AutoGluon:")
print(f"  Promedio: {ensemble_data['tn_autogluon'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_autogluon'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_autogluon'].std():.2f}")

print("\nEstadísticas de Regresión Lineal:")
print(f"  Promedio: {ensemble_data['tn_regresion'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_regresion'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_regresion'].std():.2f}")

print("\nEstadísticas del Ensemble Ponderado (40% AutoGluon + 60% Regresión):")
print(f"  Promedio: {ensemble_data['tn'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn'].median():.2f}")
print(f"  Std:      {ensemble_data['tn'].std():.2f}")

# Correlación entre modelos
correlacion = ensemble_data['tn_autogluon'].corr(ensemble_data['tn_regresion'])
print(f"\nCorrelación entre modelos: {correlacion:.4f}")

# Mostrar pesos utilizados
print(f"\nPesos aplicados:")
print(f"  AutoGluon: {peso_autogluon*100:.0f}%")
print(f"  Regresión Lineal: {peso_regresion*100:.0f}%")

print(f"\n✅ Ensemble ponderado completado exitosamente!")
print(f"Archivo final: pred_autogluon_rl_03.csv")
print(f"Total de productos: {len(resultado_final)}")

RESUMEN DEL ENSEMBLE
Estadísticas de AutoGluon:
  Promedio: 36.25
  Mediana:  9.06
  Std:      93.69

Estadísticas de Regresión Lineal:
  Promedio: 35.67
  Mediana:  9.04
  Std:      92.24

Estadísticas del Ensemble Ponderado (40% AutoGluon + 60% Regresión):
  Promedio: 35.82
  Mediana:  9.19
  Std:      92.48

Correlación entre modelos: 0.9931

Pesos aplicados:
  AutoGluon: 25%
  Regresión Lineal: 75%

✅ Ensemble ponderado completado exitosamente!
Archivo final: pred_autogluon_rl_03.csv
Total de productos: 780
